In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

import re
import time

rm_parenthesis = re.compile(r"\（.*?\）|\（ .*? \）|\(.*?\)|\( .*? \)")

## Получение датасета с пиньинем

In [ ]:
options = Options()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

try:
    driver.get("https://mandarinbean.com/new-hsk-1-word-list/")
    headers = [header.get_attribute("innerHTML") for header in driver.find_elements(By.TAG_NAME, "strong")]
    final_data = pd.DataFrame(columns=headers)

    def get_table(url):     # Берёт страницу и собирает с неё все элементы таблицы (в тегах <td>) в датафрейм

        def divide_rows(row):   # Разделяет один ряд с n-ным количеством "|" на n+1 рядов
            result = []
            while "｜" in "".join(row):
                irow = [row[0], row[1][:row[1].index("｜")], row[2][:row[2].index("｜")-1], row[3]]
                row = [row[0], row[1][row[1].index("｜")+1:], row[2][row[2].index("｜")+2:], row[3]]
                result.append(irow)
            result.append([row[0], row[1][0:], row[2][0:], row[3]])
            return result 

        driver.get(url)
        element = WebDriverWait(driver, 10).until(          # Ожидание прогрузки элементов в тегах <td>
            EC.presence_of_element_located((By.TAG_NAME, "td"))
        )

        temp_data = [value.get_attribute("innerHTML") for value in driver.find_elements(By.TAG_NAME, "td")]

        data = []
        for i in range(4, len(temp_data), 4):
            temp_arr = [re.sub(rm_parenthesis, "", td.replace("&nbsp;", " ")) for td in temp_data[i-4:i]]
            if "｜" in "".join(temp_arr):
                temp_arr = divide_rows(temp_arr)
            data.extend([temp_arr] if len(temp_arr)==4 else temp_arr)

        return pd.DataFrame(data=data, columns=headers)

    for i in range(1, 7):
        final_data = pd.concat(objs=[final_data, get_table(f"https://mandarinbean.com/new-hsk-{i}-word-list/")]) 
        
    final_data.drop(labels=["No", "English"], axis=1, inplace=True)
    final_data.to_csv(path_or_buf=".\\pinyin.csv", index=False)

except Exception as e:
    print(f"{e}\n\nThe process will terminate in 10 seconds...")
    time.sleep(10)

finally:
    driver.close()
    driver.quit()

## Скрейпинг для других систем транскрипции

### Палладий

In [ ]:
final_data = pd.read_csv("./test.csv")

driver = webdriver.Chrome()

try:
    def convert_system(
        url, 
        src_table, 
        textbox, 
        convert_button, 
        output, 
        system_selector=None, 
        variations=None
        ):

        # def clear_text(element):
        #     length = len(element.get_attribute('value'))
        #     element.send_keys(length * Keys.BACKSPACE)

        if variations is not None:
            pass

        driver.get(url)       
        scraped = []
        
        textbox = driver.find_element(By.CLASS_NAME, textbox)
        wait = WebDriverWait(driver, 10)
        wait.until(lambda w: textbox.is_displayed())

        convert_button = driver.find_element(By.CLASS_NAME, convert_button)
        output = driver.find_element(By.CLASS_NAME, output)

        for tra in src_table:
            wait.until(lambda w: textbox.is_displayed())
            textbox.send_keys(tra)
            wait.until(lambda w: convert_button.is_enabled())
            convert_button.click()
    
            time.sleep(2)
            scraped.append(output.get_attribute("innerHTML"))
            wait.until(lambda w: textbox.is_enabled())
            textbox.clear()
            # clear_text(textbox)
        return scraped
    
    palladius = pd.Series(
        data=convert_system(
            "https://palladius.ru/", 
            list(final_data["Pinyin"]),
            "pinyin",
            "translate",
            "palladius"
            ),
        name="Palladius"
        )
    final_data = pd.concat(objs=[final_data, palladius], axis=1, index=False)
    print(final_data)

except Exception as e:
    print(f"{e}\n\nThe process will terminate in 10 seconds...")
    time.sleep(10)

finally:
    driver.close()
    driver.quit()

Message: invalid element state: Element is not currently interactable and may not be manipulated
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00007FF73E2382B2+55298]
	(No symbol) [0x00007FF73E1A5E02]
	(No symbol) [0x00007FF73E0605AB]
	(No symbol) [0x00007FF73E06509C]
	(No symbol) [0x00007FF73E066FDF]
	(No symbol) [0x00007FF73E0670D0]
	(No symbol) [0x00007FF73E09A22B]
	(No symbol) [0x00007FF73E0C20AA]
	(No symbol) [0x00007FF73E0980CF]
	(No symbol) [0x00007FF73E0C22C0]
	(No symbol) [0x00007FF73E0DAAA4]
	(No symbol) [0x00007FF73E0C1E83]
	(No symbol) [0x00007FF73E09670A]
	(No symbol) [0x00007FF73E097964]
	GetHandleVerifier [0x00007FF73E5B0AAB+3694587]
	GetHandleVerifier [0x00007FF73E60728E+4048862]
	GetHandleVerifier [0x00007FF73E5FF173+4015811]
	GetHandleVerifier [0x00007FF73E2D47D6+695590]
	(No symbol) [0x00007FF73E1B0CE8]
	(No symbol) [0x00007FF73E1ACF34]
	(No symbol) [0x00007FF73E1AD062]
	(No symbol) [0x00007FF73E19D3A3]
	BaseThreadInitThunk [0x00007FFA0F8D

# AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [ ]:
final_data = pd.read_csv("./test.csv")
driver = webdriver.Chrome()

try:
    def map_table(url, src_table):
        
        result = []

        def divider(table):
            for entry in table:
                yield entry[:entry.index("—")-1], entry[entry.index("—")+2:]

        def mapper(pinyin, palladius):
            for i in range(len(table)):
                result.append(next(palladius) if next(pinyin)==)

        driver.get(url)
        table = [value.get_attribute("innerHTML") for value in driver.find_elements(By.TAG_NAME, "td")]

        return mapper(*divider(table))
            

    final_data = pd.concat([final_data, pd.Series(
        data=map_table("https://cidian.ru/palladius", list(final_data["Pinyin"])),
        name="Palladius"
    )], axis=1)
    final_data.to_csv(path_or_buf=".\\data.csv", index=False)

# except StopIteration:
#     print("Done successfully!")

except Exception as e:
    print(f"{e}\n\nThe process will terminate in 10 seconds...")
    time.sleep(10)

finally:
    driver.close()
    driver.quit()

### Система Уэйда-Джайлза и Гоюй ломацзы

In [6]:
import json
import re

final_data = pd.read_csv("./pinyin.csv")

def table_mapper(src_table:list, src_system:list, other_system:list) -> list:
    output = []
    for src_entry in src_table:
        src_entry = "".join(re.split(r"·|\s|’", src_entry.lower()))
        src_system = ["".join(re.split(r"\s", s.lower())) for s in src_system]
        if src_entry in src_system:
            output.append(other_system[src_system.index(src_entry)])
        else:
            # output.append("NOPE! :P")   # Использовано для дебага
            output.append(pd.NaT)
    return output

with open("./complete.json", mode="r", encoding="utf-8") as file:
    complete = json.load(file)
    pinyin, wadegiles, romatzyh = [], [], []
    # i = 0
    for entry in complete:
        pinyin.append(entry["forms"][0]["transcriptions"]["pinyin"])
        wadegiles.append(entry["forms"][0]["transcriptions"]["wadegiles"])    
        romatzyh.append(entry["forms"][0]["transcriptions"]["romatzyh"])
        # print(f"Pinyin: {pinyin[i]}\nWadegiles: {wadegiles[i]}\nRomatzyh: {romatzyh[i]}\n")
        # i += 1

final_data = pd.concat([final_data, pd.Series(
    data=table_mapper(list(final_data["Pinyin"]), pinyin, wadegiles),
    name="Wade-Giles")
    ], axis=1)
final_data = pd.concat([final_data, pd.Series(
    data=table_mapper(list(final_data["Pinyin"]), pinyin, romatzyh),
    name="Romatzyh")
    ], axis=1)
final_data.dropna(inplace=True)
final_data.reset_index(inplace=True, drop=True)
final_data.to_csv(path_or_buf=".\\chinese_dataset.csv", index=False)
    